In [15]:
from pyomnisci import connect
import pandas as pd
from pathlib import Path
import vegafusion_jupyter as vf

In [18]:
outdir = Path("data") / "omnisci"
outdir.mkdir(exist_ok=True)

tmp_outdir = Path("tmp_flights_donotmodify")
tmp_outdir.mkdir(exist_ok=True)

host = "metis.mapd.com"
dbname = "mapd"

In [3]:
con = connect(
    host=host, user='demouser', password='HyperInteractive',
    port=443, dbname=dbname, protocol= 'https'
)

In [4]:
# # Display all tables
# con.get_tables()

# # Display all columns
# [c.name for c in con.get_table_details(table)]

In [5]:
table = "flights_donotmodify"

cols = [
 'uniquecarrier',
 'arrdelay',
 'depdelay',
 'dep_timestamp',
 'arr_timestamp',
 'origin_state',
 'dest_state',
 'dest_lat',
 'dest_lon',
 'carrier_name',
]

partition_cols = ["uniquecarrier", "dest_state"]

In [9]:
%%time
# Get all combinations of uniquecarrier and dest_state
carriers = con.select_ipc(f"""
    SELECT uniquecarrier, dest_state
    FROM flights_donotmodify
    GROUP BY {', '.join(partition_cols)}
    """)

CPU times: user 52.6 ms, sys: 4.45 ms, total: 57.1 ms
Wall time: 329 ms


In [10]:
# data_root = Path(f"./{table}/")
# data_root.mkdir(parents=True, exist_ok=True)

In [11]:
for i, row in carriers.iterrows():
    suffix = '-'.join([f"{row[c]}" for c in partition_cols])
    path = tmp_outdir / f"{table}-{suffix}.feather"
    if not path.exists():
        condition = " AND ".join(
            [f"{c} = {repr(row[c])}" for c in partition_cols]
        )
        query = f"""
            SELECT {', '.join(cols)}
            FROM {table}
            WHERE {condition}
        """
        print(suffix)
        df = con.select_ipc(query)
        df.to_feather(path)

WN-TX
XE-TX


KeyboardInterrupt: 

In [12]:
# Read partitions
dfs = []
for i, row in carriers.iterrows():
    suffix = '-'.join([f"{row[c]}" for c in partition_cols])
    path = tmp_outdir / f"{table}-{suffix}.feather"
    dfs.append(pd.read_feather(path))

In [ ]:
flights = pd.concat(dfs, axis=0).reset_index()

vf.to_feather(flights, outdir / f"{table}.feather")